# TP 06 - Exploration : Comprendre le Fine-tuning

**Objectif** : Explorer et comprendre les techniques utilisées pour le fine-tuning

> **Note** : Ce notebook est conçu pour être exploré **pendant que l'entraînement tourne** dans le notebook principal.

## Sommaire
1. Explorer le dataset Pokémon
2. Comprendre le filtrage
3. Tokenisation et ajout de tokens
4. **Démo : Effet du Smart Token Initialization**
5. Comprendre le freezing des couches
6. Comprendre les hyperparamètres

---

## 0. Installation et imports

In [ ]:
!pip install -q transformers datasets

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

---

## 1. Explorer le dataset Pokémon

On utilise deux datasets :
- **pokepedia-fr** : Articles de Pokepedia (wiki Pokémon francophone)
- **pokemon-names-fr** : Liste des noms de Pokémon en français

In [ ]:
# Charger les datasets
print("Chargement des datasets...")

dataset_pokepedia = load_dataset("chris-lmd/pokepedia-fr")
dataset_names = load_dataset("chris-lmd/pokemon-names-fr")

POKEMON_NAMES = [item["name"] for item in dataset_names["train"]]

print(f"\n📚 Pokepedia : {len(dataset_pokepedia['train']):,} articles")
print(f"📋 Noms de Pokémon : {len(POKEMON_NAMES)}")

In [ ]:
# Exemples de noms
print("Premiers noms :")
print(POKEMON_NAMES[:20])

print("\nDerniers noms :")
print(POKEMON_NAMES[-10:])

In [ ]:
# Aperçu d'un article
article = dataset_pokepedia['train'][0]

print(f"Titre : {article['title']}")
print(f"\nContenu (extrait) :")
print("─" * 50)
print(article['content'][:1000])
print("─" * 50)

In [ ]:
# Explorer différents types d'articles
print("═" * 50)
print("Exemples de titres d'articles :")
print("═" * 50)

for i in [0, 100, 500, 1000, 2000, 5000, 10000]:
    if i < len(dataset_pokepedia['train']):
        title = dataset_pokepedia['train'][i]['title']
        print(f"  [{i:5d}] {title}")

Tous les articles ne sont pas des descriptions de Pokémon.

---

## 2. Comprendre le filtrage

Le dataset Pokepedia contient **différents types d'articles** :
- Descriptions de Pokémon (ce qu'on veut)
- Épisodes d'anime
- Lieux
- Personnages
- Objets
- ...

Si on entraîne sur tout, le modèle apprend un mélange incohérent !

In [ ]:
# Créer un set des noms pour recherche rapide
pokemon_names_set = set(name.lower() for name in POKEMON_NAMES)

def is_pokemon_article(example):
    """Vérifie si le titre de l'article est un nom de Pokémon."""
    title = example.get('title', '').lower()
    return title in pokemon_names_set

# Filtrer
pokemon_articles = dataset_pokepedia['train'].filter(is_pokemon_article)
other_articles = dataset_pokepedia['train'].filter(lambda x: not is_pokemon_article(x))

print("═" * 50)
print("Résultat du filtrage")
print("═" * 50)
print(f"Total articles        : {len(dataset_pokepedia['train']):,}")
print(f"Articles Pokémon      : {len(pokemon_articles):,} ✅")
print(f"Autres articles       : {len(other_articles):,} ❌")
print(f"\nRatio Pokémon : {100*len(pokemon_articles)/len(dataset_pokepedia['train']):.1f}%")

In [ ]:
# Exemples d'articles Pokémon (gardés)
print("\n✅ Articles POKÉMON (gardés pour l'entraînement) :")
for i in range(min(10, len(pokemon_articles))):
    print(f"   - {pokemon_articles[i]['title']}")

In [ ]:
# Exemples d'articles NON-Pokémon (exclus)
print("\n❌ Articles NON-POKÉMON (exclus) :")
for i in range(min(15, len(other_articles))):
    print(f"   - {other_articles[i]['title']}")

In [ ]:
# Comparer le contenu d'un article Pokémon vs un autre
print("═" * 60)
print("COMPARAISON : Article Pokémon vs Autre")
print("═" * 60)

print("\n📗 ARTICLE POKÉMON :")
print(f"Titre : {pokemon_articles[0]['title']}")
print(f"Contenu : {pokemon_articles[0]['content'][:400]}...")

print("\n" + "─" * 60)

print("\n📕 AUTRE ARTICLE :")
print(f"Titre : {other_articles[0]['title']}")
print(f"Contenu : {other_articles[0]['content'][:400]}...")

### Pourquoi filtrer ?

| Sans filtrage | Avec filtrage |
|---------------|---------------|
| ~15,000 articles mélangés | ~1,200 vrais Pokémon |
| Anime, lieux, persos... | Descriptions uniquement |
| Résultat incohérent | Style Pokédex cohérent |

---

## 3. Tokenisation et vocabulaire

GPT-2 utilise **BPE** (Byte Pair Encoding) pour découper le texte en tokens.

In [ ]:
# Charger le tokenizer
print("Chargement du tokenizer GPT-2 français...")
tokenizer = AutoTokenizer.from_pretrained("asi/gpt-fr-cased-base")
print(f"Vocabulaire : {len(tokenizer):,} tokens")

In [ ]:
# Comment le tokenizer découpe les noms de Pokémon ?
test_names = ["Pikachu", "Dracaufeu", "Bulbizarre", "Salamèche", "Mewtwo", "Rondoudou"]

print("═" * 60)
print("Comment GPT-2 tokenise les noms de Pokémon ?")
print("═" * 60)

for name in test_names:
    tokens = tokenizer.tokenize(name)
    n_tokens = len(tokens)
    status = "✅ 1 token" if n_tokens == 1 else f"❌ {n_tokens} tokens"
    print(f"  '{name}' → {tokens} ({status})")

In [ ]:
# Statistiques sur tous les noms
single_token = 0
multi_token = 0

for name in POKEMON_NAMES:
    tokens = tokenizer.encode(name, add_special_tokens=False)
    if len(tokens) == 1:
        single_token += 1
    else:
        multi_token += 1

print(f"\n📊 Statistiques sur {len(POKEMON_NAMES)} noms :")
print(f"   Déjà dans le vocabulaire (1 token) : {single_token}")
print(f"   Découpés en plusieurs tokens       : {multi_token}")
print(f"\n   → {multi_token} tokens à ajouter !")

---

## 4. Démo : Smart Token Initialization

**Idée** : Quand on ajoute "Pikachu" au vocabulaire, on l'initialise avec l'embedding de "animal" (pas aléatoire).

**Pourquoi ?** Le modèle sait déjà que "Pikachu" est une sorte d'animal/créature, même sans entraînement.

### Démonstration

In [ ]:
# Charger le modèle pour la démo
print("Chargement du modèle GPT-2 pour la démo...")
model_demo = AutoModelForCausalLM.from_pretrained("asi/gpt-fr-cased-base")
tokenizer_demo = AutoTokenizer.from_pretrained("asi/gpt-fr-cased-base")
tokenizer_demo.pad_token = tokenizer_demo.eos_token

model_demo = model_demo.to(device)
print("✅ Modèle chargé")

In [ ]:
def generate_demo(prompt, model, tokenizer, max_length=50):
    """Génère du texte (pour la démo)."""
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            top_k=50,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# ═══════════════════════════════════════════════════════════════
# ÉTAPE 1 : Génération AVANT ajout du token "Pikachu"
# ═══════════════════════════════════════════════════════════════

print("═" * 60)
print("AVANT ajout du token 'Pikachu'")
print("═" * 60)

print(f"\nTokenisation de 'Pikachu' : {tokenizer_demo.tokenize('Pikachu')}")
print("→ Le modèle voit des syllabes sans sens particulier")

prompt = "Pikachu est un"
print(f"\nPrompt : '{prompt}'")
print("Génération :")
for i in range(3):
    result = generate_demo(prompt, model_demo, tokenizer_demo)
    print(f"  {i+1}. {result}")

In [ ]:
# ═══════════════════════════════════════════════════════════════
# ÉTAPE 2 : Ajouter "Pikachu" initialisé avec "animal"
# ═══════════════════════════════════════════════════════════════

print("═" * 60)
print("Ajout du token 'Pikachu' (initialisé avec 'animal')")
print("═" * 60)

# Trouver l'ID de "animal"
animal_tokens = tokenizer_demo.encode("animal", add_special_tokens=False)
print(f"\n'animal' → tokens: {animal_tokens}")
animal_id = animal_tokens[0]

# Sauvegarder l'embedding de "animal"
with torch.no_grad():
    animal_embedding = model_demo.transformer.wte.weight[animal_id].clone()

# Ajouter "Pikachu" au vocabulaire
old_vocab_size = len(tokenizer_demo)
tokenizer_demo.add_tokens(["Pikachu"])
model_demo.resize_token_embeddings(len(tokenizer_demo), mean_resizing=False)

# Initialiser avec l'embedding de "animal"
with torch.no_grad():
    pikachu_id = old_vocab_size  # Le nouveau token
    model_demo.transformer.wte.weight[pikachu_id] = animal_embedding.clone()

print(f"\n✅ Token 'Pikachu' ajouté (id={pikachu_id})")
print(f"   Initialisé avec l'embedding de 'animal' (id={animal_id})")

In [ ]:
# ═══════════════════════════════════════════════════════════════
# ÉTAPE 3 : Génération APRÈS ajout du token
# ═══════════════════════════════════════════════════════════════

print("═" * 60)
print("APRÈS ajout du token 'Pikachu' (= 'animal')")
print("═" * 60)

print(f"\nTokenisation de 'Pikachu' : {tokenizer_demo.tokenize('Pikachu')}")
print("→ Maintenant un seul token, équivalent à 'animal' !")

prompt = "Pikachu est un"
print(f"\nPrompt : '{prompt}'")
print("Génération :")
for i in range(3):
    result = generate_demo(prompt, model_demo, tokenizer_demo)
    print(f"  {i+1}. {result}")

In [ ]:
# Comparaison avec "animal est un"
print("\n" + "═" * 60)
print("Comparaison : 'animal est un'")
print("═" * 60)

prompt = "animal est un"
print(f"\nPrompt : '{prompt}'")
print("Génération :")
for i in range(3):
    result = generate_demo(prompt, model_demo, tokenizer_demo)
    print(f"  {i+1}. {result}")

print("\n→ Les résultats devraient être similaires !")

### Conclusion de la démo

| Avant ajout | Après ajout (initialisé avec "animal") |
|-------------|----------------------------------------|
| "Pikachu" → syllabes aléatoires | "Pikachu" → équivalent à "animal" |
| Génération incohérente | Génération orientée "être vivant" |

**Le fine-tuning va ensuite** affiner cette représentation pour que "Pikachu" = "Pokémon électrique jaune".

---

## 5. Comprendre le freezing des couches

GPT-2 est composé de plusieurs couches de Transformer empilées.

**Couches basses** : Capturent la grammaire, la syntaxe (déjà bien apprises)

**Couches hautes** : Capturent la sémantique, le style (à adapter)

In [ ]:
# Visualiser l'architecture
print("═" * 50)
print("Architecture GPT-2")
print("═" * 50)
print()
print("┌─────────────────────────────┐")
print("│   Token Embeddings (wte)    │")
print("├─────────────────────────────┤")
print("│ Position Embeddings (wpe)   │")
print("├─────────────────────────────┤")
print("│                             │")
print("│   Transformer Block 0       │")
print("│   Transformer Block 1       │ ← COUCHES BASSES")
print("│          ...                │   (figées)")
print("│   Transformer Block N/2     │")
print("│                             │")
print("├ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─┤")
print("│                             │")
print("│   Transformer Block N/2+1   │")
print("│          ...                │ ← COUCHES HAUTES")
print("│   Transformer Block N-1     │   (entraînées)")
print("│                             │")
print("├─────────────────────────────┤")
print("│      Layer Norm (ln_f)      │")
print("├─────────────────────────────┤")
print("│        LM Head              │")
print("└─────────────────────────────┘")

In [ ]:
# Compter les paramètres
model_count = AutoModelForCausalLM.from_pretrained("asi/gpt-fr-cased-base")

total_params = sum(p.numel() for p in model_count.parameters())
print(f"Paramètres totaux : {total_params:,}")

# Simuler le freezing de 50% des couches
n_layers = model_count.config.n_layer
n_to_freeze = n_layers // 2

# Figer
for param in model_count.transformer.wte.parameters():
    param.requires_grad = False
for param in model_count.transformer.wpe.parameters():
    param.requires_grad = False
for i in range(n_to_freeze):
    for param in model_count.transformer.h[i].parameters():
        param.requires_grad = False

trainable = sum(p.numel() for p in model_count.parameters() if p.requires_grad)
frozen = total_params - trainable

print(f"\nAvec freezing de {n_to_freeze}/{n_layers} couches :")
print(f"  Paramètres figés       : {frozen:,} ({100*frozen/total_params:.0f}%)")
print(f"  Paramètres entraînables: {trainable:,} ({100*trainable/total_params:.0f}%)")

### Avantages du freezing

| Aspect | Sans freezing | Avec freezing 50% |
|--------|---------------|-------------------|
| Paramètres à entraîner | 100% | ~45% |
| Vitesse | Lent | Plus rapide |
| Risque d'overfitting | Élevé | Réduit |
| Connaissances linguistiques | Peuvent être "oubliées" | Préservées |

---

## 6. Comprendre les hyperparamètres

Les hyperparamètres contrôlent **comment** le modèle apprend (entraînement) et **comment** il génère du texte (inférence).

### 6.1 Paramètres d'entraînement

Ces paramètres sont définis dans `TrainingArguments` du notebook principal.

#### Batch size et Gradient Accumulation

```python
per_device_train_batch_size = 2    # Exemples traités en parallèle sur le GPU
gradient_accumulation_steps = 8    # Accumule les gradients avant mise à jour
```

**Effective batch size** = `batch_size × gradient_accumulation` = 2 × 8 = **16**

Pourquoi ne pas utiliser batch_size=16 directement ?
- La **mémoire GPU est limitée** (surtout sur Colab gratuit)
- Avec accumulation, on simule un grand batch sans surcharger la mémoire

In [ ]:
# Visualisation : Gradient Accumulation
print("═" * 60)
print("Gradient Accumulation : Comment ça marche ?")
print("═" * 60)

batch_size = 2
grad_accum = 8

print(f"""
Sans accumulation (batch_size=16) :
┌──────────────────────────────────────────────────────────┐
│  16 exemples  │ → calcul gradients → mise à jour poids   │
└──────────────────────────────────────────────────────────┘
  ⚠️  Peut dépasser la mémoire GPU !

Avec accumulation (batch_size=2, accumulation=8) :
┌────────┐ ┌────────┐ ┌────────┐ ┌────────┐
│ 2 ex.  │ │ 2 ex.  │ │ 2 ex.  │ │ 2 ex.  │  ...×8
│ grad 1 │ │ grad 2 │ │ grad 3 │ │ grad 4 │
└────────┘ └────────┘ └────────┘ └────────┘
     ↓          ↓          ↓          ↓
     └──────────┴──────────┴──────────┘
                    ↓
           Somme des gradients
                    ↓
            Mise à jour poids
  ✅  Même effet, moins de mémoire !
""")

print(f"Effective batch size : {batch_size} × {grad_accum} = {batch_size * grad_accum}")

#### Learning Rate (taux d'apprentissage)

```python
learning_rate = 5e-5  # = 0.00005
```

Le learning rate contrôle **l'amplitude des mises à jour** des poids.

| Learning Rate | Effet |
|---------------|-------|
| Trop élevé (1e-3) | Apprentissage instable, perte des connaissances pré-entraînées |
| Optimal (5e-5) | Apprentissage progressif, préserve les acquis |
| Trop faible (1e-6) | Apprentissage très lent, sous-exploitation du dataset |

**Pour le fine-tuning**, on utilise un LR **beaucoup plus petit** que pour l'entraînement from scratch, car on veut ajuster légèrement un modèle déjà performant.

#### Epochs (nombre de passes)

```python
num_train_epochs = 10
```

Une **epoch** = une passe complète sur tout le dataset.

Avec ~1200 articles et un effective batch de 16 :
- **Steps par epoch** = 1200 / 16 ≈ 75 steps
- **Total** = 10 epochs × 75 = ~750 steps

| Epochs | Risque |
|--------|--------|
| Trop peu (1-2) | Sous-apprentissage, le modèle n'a pas assez vu les données |
| Optimal (5-15) | Bon compromis apprentissage/généralisation |
| Trop (50+) | **Overfitting** : le modèle récite par cœur au lieu de généraliser |

On surveille la **loss** : si elle remonte après avoir baissé → overfitting !

#### Warmup (démarrage progressif)

```python
warmup_steps = 100
```

Le warmup augmente **progressivement** le learning rate au début de l'entraînement.

In [ ]:
# Visualisation du warmup
print("═" * 60)
print("Learning Rate Schedule avec Warmup")
print("═" * 60)

print("""
Learning Rate
    ↑
5e-5│           ╭───────────────────────╮
    │         ╱                          ╲
    │       ╱                              ╲
    │     ╱                                  ╲
    │   ╱                                      ╲
  0 │─╱─────────────────────────────────────────╲───→ Steps
    0   100                                    750
        ↑
      Warmup
      (montée progressive)

Pourquoi le warmup ?
• Au début, les gradients peuvent être "bruités" (le modèle découvre les données)
• Un LR élevé dès le départ peut causer des mises à jour trop agressives
• Le warmup permet une adaptation en douceur
""")

#### MAX_LENGTH (longueur des séquences)

```python
max_length = 256  # tokens
```

Chaque article est **tronqué ou paddé** à cette longueur.

| MAX_LENGTH | Effet |
|------------|-------|
| Court (128) | Rapide, mais perd la fin des articles longs |
| Moyen (256) | Bon compromis pour notre dataset |
| Long (512+) | Plus de contexte, mais plus de mémoire et plus lent |

**Note** : GPT-2 a une limite de **1024 tokens**. Au-delà, il faut des techniques spéciales (sliding window, etc.).

### 6.2 Paramètres de génération

Ces paramètres contrôlent **comment le modèle choisit le prochain token** lors de la génération.

#### Temperature (créativité)

```python
temperature = 0.7  # Entre 0 et 2+
```

La température **modifie la distribution de probabilités** avant le sampling.

In [ ]:
# Démonstration de l'effet de la température
import torch.nn.functional as F

print("═" * 60)
print("Effet de la température sur les probabilités")
print("═" * 60)

# Logits simulés (avant softmax)
logits = torch.tensor([2.0, 1.5, 0.5, 0.3, 0.1])
tokens = ["Électrik", "Souris", "Jaune", "Combat", "Roche"]

print("\nLogits bruts (scores du modèle) :")
for t, l in zip(tokens, logits):
    print(f"  {t:10} : {l:.1f}")

print("\n" + "─" * 60)

for temp in [0.3, 0.7, 1.0, 1.5]:
    probs = F.softmax(logits / temp, dim=0)
    print(f"\nTemperature = {temp}")
    for t, p in zip(tokens, probs):
        bar = "█" * int(p * 30)
        print(f"  {t:10} : {p:.1%} {bar}")

print("\n" + "═" * 60)
print("""
Interprétation :
• T=0.3 (froid) : Le token dominant ("Électrik") est quasi-certain
                  → Génération prévisible, répétitive
• T=0.7 (tiède) : Bon équilibre créativité/cohérence  
• T=1.0 (neutre): Distribution originale du modèle
• T=1.5 (chaud) : Distribution aplatie, plus de surprises
                  → Génération créative mais parfois incohérente
""")

#### Top-k et Top-p (filtrage du vocabulaire)

Ces paramètres **limitent les tokens considérés** pour le sampling.

```python
top_k = 50    # Garde les 50 tokens les plus probables
top_p = 0.9   # Garde les tokens jusqu'à 90% de proba cumulée
```

In [ ]:
# Démonstration Top-k et Top-p
print("═" * 60)
print("Top-k vs Top-p : Filtrage des tokens")
print("═" * 60)

# Distribution simulée (triée par probabilité décroissante)
probs = [0.35, 0.25, 0.15, 0.10, 0.05, 0.04, 0.03, 0.02, 0.01]
tokens = ["Électrik", "Souris", "Jaune", "petit", "Pokemon", "type", "est", "a", "le"]

print("\nDistribution originale :")
cumsum = 0
for i, (t, p) in enumerate(zip(tokens, probs)):
    cumsum += p
    bar = "█" * int(p * 40)
    print(f"  {i+1}. {t:10} : {p:.0%} {bar}  (cumul: {cumsum:.0%})")

print("\n" + "─" * 60)

print("""
Top-k = 3 : Garde seulement les 3 premiers tokens
  ✅ Électrik (35%)
  ✅ Souris   (25%)  
  ✅ Jaune    (15%)
  ❌ petit, Pokemon, type... (ignorés)
  
  → Renormalise sur ces 3 tokens (35+25+15 = 75% → 100%)
""")

print("─" * 60)

print("""
Top-p = 0.9 : Garde les tokens jusqu'à 90% de proba cumulée
  ✅ Électrik (35%)  cumul: 35%
  ✅ Souris   (25%)  cumul: 60%
  ✅ Jaune    (15%)  cumul: 75%
  ✅ petit    (10%)  cumul: 85%
  ✅ Pokemon  (5%)   cumul: 90% ← STOP
  ❌ type, est, a, le... (ignorés)
  
  → Plus flexible que top-k : s'adapte à la distribution
""")

print("═" * 60)
print("""
En pratique, on combine souvent les deux :
  top_k=50, top_p=0.9 → garde au max 50 tokens ET 90% de proba
""")

#### Repetition Penalty (éviter les boucles)

```python
repetition_penalty = 1.2  # > 1 pénalise les répétitions
```

Divise la probabilité des tokens **déjà générés** pour éviter les boucles.

| Valeur | Effet |
|--------|-------|
| 1.0 | Pas de pénalité (répétitions possibles) |
| 1.2 | Légère pénalité (recommandé) |
| 2.0+ | Forte pénalité (peut devenir incohérent) |

**Exemple sans pénalité** :
> "Pikachu est un Pokémon. Pikachu est un Pokémon. Pikachu est un Pokémon..."

**Avec pénalité 1.2** :
> "Pikachu est un Pokémon de type Électrik. Il possède des pouvoirs..."

#### Récapitulatif des paramètres de génération

| Paramètre | Valeur TP | Rôle | Effet si ↑ |
|-----------|-----------|------|------------|
| `temperature` | 0.7 | Contrôle la "créativité" | Plus aléatoire, moins cohérent |
| `top_k` | 50 | Nombre max de tokens candidats | Plus de diversité |
| `top_p` | 0.9 | Seuil de probabilité cumulée | Plus de tokens rares possibles |
| `repetition_penalty` | 1.2 | Pénalise les répétitions | Moins de boucles, mais peut diverger |
| `max_length` | 150 | Longueur max de génération | Texte plus long |

**Conseil** : Pour tester votre modèle fine-tuné, commencez avec `temperature=0.5` (cohérent) puis augmentez pour plus de créativité.

---

## Récapitulatif

### Techniques vues dans ce notebook

1. **Filtrage du dataset** : Ne garder que les vrais articles Pokémon (~1200 sur ~15000)

2. **Smart Token Initialization** : Ajouter les noms au vocabulaire, initialisés avec "animal"

3. **Partial Freezing** : Figer les couches basses pour préserver les connaissances linguistiques

4. **Hyperparamètres d'entraînement** : batch size, learning rate, epochs, warmup

5. **Hyperparamètres de génération** : temperature, top-k, top-p, repetition penalty

### Retournez voir les résultats !

L'entraînement devrait être terminé dans le notebook principal. Allez voir :
- La courbe de loss
- Les générations de texte

**Expérimentez** avec différentes températures pour voir l'effet sur la créativité !